# Exemplu de utilizare a instrumentului AutoGen


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## Crearea Clientului

În acest exemplu, vom utiliza [Modelele GitHub](https://aka.ms/ai-agents-beginners/github-models) pentru a accesa LLM-ul.

`model` este definit ca `gpt-4o-mini`. Încercați să schimbați modelul cu un alt model disponibil pe piața de modele GitHub pentru a vedea rezultate diferite.

Ca un test rapid, vom rula doar un prompt simplu - `Care este capitala Franței`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Definirea Funcțiilor

În acest exemplu, vom oferi agentului acces la un instrument care este o funcție cu o listă de destinații de vacanță disponibile și disponibilitatea acestora.

Te poți gândi că acesta ar fi un scenariu în care un agent de turism ar putea avea acces, de exemplu, la o bază de date de călătorii.

Pe măsură ce parcurgi acest exemplu, simte-te liber să încerci să definești funcții și instrumente noi pe care agentul le poate apela.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## Definirea Instrumentului Funcție  
Pentru ca agentul să utilizeze `vacation_destinations` ca un `FunctionTool`, trebuie să îl definim ca atare.  

Vom oferi, de asemenea, o descriere a instrumentului, care este utilă pentru ca agentul să identifice la ce este folosit acest instrument în raport cu sarcina solicitată de utilizator.  


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Definirea agentului

Acum putem crea agentul în codul de mai jos. Definim `system_message` pentru a oferi agentului instrucțiuni despre cum să ajute utilizatorii să găsească destinații de vacanță.

De asemenea, setăm parametrul `reflect_on_tool_use` la true. Acest lucru permite utilizarea LLM pentru a prelua răspunsul apelului instrumentului și a trimite răspunsul folosind limbaj natural.

Poți seta parametrul la false pentru a vedea diferența.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## Rularea Agentului

Acum putem rula agentul cu mesajul inițial al utilizatorului, cerând să facă o excursie la Tokyo.

Puteți schimba destinația orașului pentru a vedea cum răspunde agentul în funcție de disponibilitatea orașului.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**Declinare de responsabilitate**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim să asigurăm acuratețea, vă rugăm să rețineți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa natală ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm responsabilitatea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
